<a href="https://colab.research.google.com/github/Saver404/my-demo/blob/main/2312RES1016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import math
import random
from copy import deepcopy
from typing import List, Tuple, Optional

class PuzzleState:
    def __init__(self, board: List[List[str]]):
        self.board = board
        self.size = 3
        self.blank_pos = self.find_blank()

    def find_blank(self) -> Tuple[int, int]:
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 'B':
                    return (i, j)
        return (-1, -1)

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(str(self.board))

    def __str__(self):
        return '\n'.join([' '.join(row) for row in self.board])

    def copy(self):
        return PuzzleState(deepcopy(self.board))

class EightPuzzle:
    def __init__(self, start_state: List[List[str]], goal_state: List[List[str]]):
        self.start = PuzzleState(start_state)
        self.goal = PuzzleState(goal_state)
        self.directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up

    def get_goal_position(self, value: str) -> Tuple[int, int]:
        """Get the goal position for a given value"""
        for i in range(3):
            for j in range(3):
                if self.goal.board[i][j] == value:
                    return (i, j)
        return (-1, -1)

    def heuristic_misplaced_tiles(self, state: PuzzleState) -> int:
        """h₁(n): Number of misplaced tiles"""
        misplaced = 0
        for i in range(3):
            for j in range(3):
                if state.board[i][j] != 'B' and state.board[i][j] != self.goal.board[i][j]:
                    misplaced += 1
        return misplaced

    def heuristic_manhattan(self, state: PuzzleState) -> int:
        """h₂(n): Total Manhattan distance"""
        distance = 0
        for i in range(3):
            for j in range(3):
                if state.board[i][j] != 'B':
                    goal_i, goal_j = self.get_goal_position(state.board[i][j])
                    distance += abs(i - goal_i) + abs(j - goal_j)
        return distance

    def get_neighbors(self, state: PuzzleState) -> List[PuzzleState]:
        """Generate all possible neighbor states"""
        neighbors = []
        i, j = state.blank_pos

        for dx, dy in self.directions:
            new_i, new_j = i + dx, j + dy
            if 0 <= new_i < 3 and 0 <= new_j < 3:
                new_board = deepcopy(state.board)
                # Swap blank with adjacent tile
                new_board[i][j], new_board[new_i][new_j] = new_board[new_i][new_j], new_board[i][j]
                neighbors.append(PuzzleState(new_board))

        return neighbors

    def is_goal(self, state: PuzzleState) -> bool:
        return state == self.goal

class HillClimbing:
    def __init__(self, puzzle: EightPuzzle, heuristic_func: str = 'manhattan'):
        self.puzzle = puzzle
        self.heuristic_func = getattr(puzzle, f'heuristic_{heuristic_func}')
        self.states_explored = 0
        self.optimal_path = []
        self.execution_time = 0

    def solve(self, max_iterations: int = 1000) -> dict:
        """Implement Hill Climbing algorithm"""
        start_time = time.time()
        current_state = self.puzzle.start.copy()
        current_cost = self.heuristic_func(current_state)

        self.states_explored = 0
        self.optimal_path = [current_state]

        for iteration in range(max_iterations):
            self.states_explored += 1

            if self.puzzle.is_goal(current_state):
                self.execution_time = time.time() - start_time
                return self._success_result()

            # Get all neighbors and their costs
            neighbors = self.puzzle.get_neighbors(current_state)
            best_neighbor = None
            best_cost = float('inf')

            for neighbor in neighbors:
                neighbor_cost = self.heuristic_func(neighbor)
                if neighbor_cost < best_cost:
                    best_cost = neighbor_cost
                    best_neighbor = neighbor

            # If no better neighbor found, stuck in local optimum
            if best_cost >= current_cost:
                self.execution_time = time.time() - start_time
                return self._failure_result(current_state)

            current_state = best_neighbor
            current_cost = best_cost
            self.optimal_path.append(current_state)

        self.execution_time = time.time() - start_time
        return self._failure_result(current_state)

    def _success_result(self) -> dict:
        return {
            'success': True,
            'states_explored': self.states_explored,
            'path_length': len(self.optimal_path),
            'path_cost': len(self.optimal_path) - 1,
            'execution_time': self.execution_time,
            'final_heuristic': self.heuristic_func(self.optimal_path[-1]),
            'optimal_path': self.optimal_path
        }

    def _failure_result(self, final_state: PuzzleState) -> dict:
        return {
            'success': False,
            'states_explored': self.states_explored,
            'path_length': len(self.optimal_path),
            'execution_time': self.execution_time,
            'final_heuristic': self.heuristic_func(final_state),
            'optimal_path': self.optimal_path
        }

class SimulatedAnnealing:
    def __init__(self, puzzle: EightPuzzle, heuristic_func: str = 'manhattan'):
        self.puzzle = puzzle
        self.heuristic_func = getattr(puzzle, f'heuristic_{heuristic_func}')
        self.states_explored = 0
        self.optimal_path = []
        self.execution_time = 0

    def cooling_schedule(self, temp: float, iteration: int) -> float:
        """Exponential cooling schedule"""
        return temp * 0.95

    def acceptance_probability(self, current_cost: float, neighbor_cost: float, temp: float) -> float:
        """Calculate acceptance probability"""
        if neighbor_cost < current_cost:
            return 1.0
        return math.exp((current_cost - neighbor_cost) / temp)

    def solve(self, initial_temp: float = 1000, max_iterations: int = 5000) -> dict:
        """Implement Simulated Annealing algorithm"""
        start_time = time.time()
        current_state = self.puzzle.start.copy()
        current_cost = self.heuristic_func(current_state)

        self.states_explored = 0
        self.optimal_path = [current_state]
        temperature = initial_temp

        best_state = current_state
        best_cost = current_cost

        for iteration in range(max_iterations):
            self.states_explored += 1
            temperature = self.cooling_schedule(temperature, iteration)

            if temperature < 1e-8:  # Stop if temperature is too low
                break

            if self.puzzle.is_goal(current_state):
                self.execution_time = time.time() - start_time
                return self._success_result()

            # Get random neighbor
            neighbors = self.puzzle.get_neighbors(current_state)
            next_state = random.choice(neighbors)
            next_cost = self.heuristic_func(next_state)

            # Update best solution found so far
            if next_cost < best_cost:
                best_state = next_state
                best_cost = next_cost

            # Decide whether to move to neighbor
            acceptance_prob = self.acceptance_probability(current_cost, next_cost, temperature)

            if acceptance_prob > random.random():
                current_state = next_state
                current_cost = next_cost
                self.optimal_path.append(current_state)

        self.execution_time = time.time() - start_time

        # Return best solution found
        if self.puzzle.is_goal(best_state):
            return self._success_result()
        else:
            return self._failure_result(best_state)

    def _success_result(self) -> dict:
        return {
            'success': True,
            'states_explored': self.states_explored,
            'path_length': len(self.optimal_path),
            'path_cost': len(self.optimal_path) - 1,
            'execution_time': self.execution_time,
            'final_heuristic': self.heuristic_func(self.optimal_path[-1]),
            'optimal_path': self.optimal_path
        }

    def _failure_result(self, final_state: PuzzleState) -> dict:
        return {
            'success': False,
            'states_explored': self.states_explored,
            'path_length': len(self.optimal_path),
            'execution_time': self.execution_time,
            'final_heuristic': self.heuristic_func(final_state),
            'optimal_path': self.optimal_path
        }

def print_results(algorithm_name: str, result: dict, puzzle: EightPuzzle):
    """Print formatted results"""
    print(f"\n{'='*60}")
    print(f"{algorithm_name} RESULTS")
    print(f"{'='*60}")

    print(f"Start State:\n{puzzle.start}")
    print(f"\nGoal State:\n{puzzle.goal}")

    if result['success']:
        print(f"\n✓ SUCCESS - Goal state reached!")
        print(f"Total states explored: {result['states_explored']}")
        print(f"States in optimal path: {result['path_length']}")
        print(f"Optimal path cost: {result['path_cost']}")
        print(f"Execution time: {result['execution_time']:.4f} seconds")
        print(f"Final heuristic value: {result['final_heuristic']}")

        print(f"\nOptimal Path (first 5 and last 5 states):")
        path = result['optimal_path']
        for i, state in enumerate(path[:5]):
            print(f"Step {i}:\n{state}\n")
        if len(path) > 10:
            print("...")
            for i, state in enumerate(path[-5:], len(path)-5):
                print(f"Step {i}:\n{state}\n")
    else:
        print(f"\n✗ FAILURE - Goal state not reached")
        print(f"Total states explored: {result['states_explored']}")
        print(f"Execution time: {result['execution_time']:.4f} seconds")
        print(f"Final heuristic value: {result['final_heuristic']}")
        print(f"Best state reached:\n{result['optimal_path'][-1]}")

def run_experiments():
    """Run multiple experiments with different configurations"""
    # Define start and goal states
    start_state = [
        ['5', 'B', '8'],
        ['4', '2', '1'],
        ['7', '3', '6']
    ]

    goal_state = [
        ['1', '2', '3'],
        ['4', '5', '6'],
        ['7', '8', 'B']
    ]

    puzzle = EightPuzzle(start_state, goal_state)

    # Experiment configurations
    experiments = [
        {'heuristic': 'manhattan', 'random_seed': 42},
        {'heuristic': 'misplaced_tiles', 'random_seed': 123},
        {'heuristic': 'manhattan', 'random_seed': 456}
    ]

    hc_results = []
    sa_results = []

    print("8-PUZZLE PROBLEM: HILL CLIMBING vs SIMULATED ANNEALING")
    print("=" * 80)

    for i, config in enumerate(experiments, 1):
        print(f"\n\nEXPERIMENT {i}")
        print(f"Heuristic: {config['heuristic']}, Random Seed: {config['random_seed']}")
        print("-" * 50)

        random.seed(config['random_seed'])

        # Hill Climbing
        hc = HillClimbing(puzzle, config['heuristic'])
        hc_result = hc.solve(max_iterations=1000)
        hc_results.append(hc_result)
        print_results("HILL CLIMBING", hc_result, puzzle)

        # Simulated Annealing
        sa = SimulatedAnnealing(puzzle, config['heuristic'])
        sa_result = sa.solve(initial_temp=1000, max_iterations=5000)
        sa_results.append(sa_result)
        print_results("SIMULATED ANNEALING", sa_result, puzzle)

    # Comparative Analysis
    print("\n\n" + "="*80)
    print("COMPARATIVE ANALYSIS")
    print("="*80)

    print("\nHILL CLIMBING SUMMARY:")
    hc_success = sum(1 for r in hc_results if r['success'])
    print(f"Success rate: {hc_success}/{len(hc_results)}")
    print(f"Average states explored: {sum(r['states_explored'] for r in hc_results) / len(hc_results):.0f}")
    print(f"Average execution time: {sum(r['execution_time'] for r in hc_results) / len(hc_results):.4f}s")

    print("\nSIMULATED ANNEALING SUMMARY:")
    sa_success = sum(1 for r in sa_results if r['success'])
    print(f"Success rate: {sa_success}/{len(sa_results)}")
    print(f"Average states explored: {sum(r['states_explored'] for r in sa_results) / len(sa_results):.0f}")
    print(f"Average execution time: {sum(r['execution_time'] for r in sa_results) / len(sa_results):.4f}s")

    # Detailed Analysis
    print("\n" + "="*80)
    print("DETAILED ANALYSIS")
    print("="*80)

    print("\n1. LOCAL OPTIMA ANALYSIS:")
    print("   Hill Climbing often gets stuck in local optima where no neighbor")
    print("   provides improvement, even though better solutions exist.")
    print("   Simulated Annealing can escape local optima by occasionally")
    print("   accepting worse solutions based on temperature.")

    print("\n2. CONVERGENCE BEHAVIOR:")
    print("   - HC: Fast initial convergence but may plateau early")
    print("   - SA: Slower convergence but continues searching globally")

    print("\n3. ROBUSTNESS:")
    print("   - HC: Highly dependent on initial state, prone to getting stuck")
    print("   - SA: More robust due to probabilistic nature")

    print("\n4. EFFICIENCY:")
    print("   - HC: Fewer states explored per iteration, faster per step")
    print("   - SA: More states explored, but better solution quality")

    print("\nCONCLUSION:")
    if sa_success > hc_success:
        print("✓ Simulated Annealing performs better for this problem due to its")
        print("  ability to escape local optima and explore the search space more thoroughly.")
    else:
        print("○ Both algorithms show comparable performance, but Hill Climbing")
        print("  may be sufficient for simpler instances with good initial states.")

if __name__ == "__main__":
    run_experiments()

8-PUZZLE PROBLEM: HILL CLIMBING vs SIMULATED ANNEALING


EXPERIMENT 1
Heuristic: manhattan, Random Seed: 42
--------------------------------------------------

HILL CLIMBING RESULTS
Start State:
5 B 8
4 2 1
7 3 6

Goal State:
1 2 3
4 5 6
7 8 B

✗ FAILURE - Goal state not reached
Total states explored: 5
Execution time: 0.0003 seconds
Final heuristic value: 9
Best state reached:
5 8 1
4 2 6
7 B 3

SIMULATED ANNEALING RESULTS
Start State:
5 B 8
4 2 1
7 3 6

Goal State:
1 2 3
4 5 6
7 8 B

✗ FAILURE - Goal state not reached
Total states explored: 494
Execution time: 0.0127 seconds
Final heuristic value: 9
Best state reached:
2 1 B
6 4 5
8 3 7


EXPERIMENT 2
Heuristic: misplaced_tiles, Random Seed: 123
--------------------------------------------------

HILL CLIMBING RESULTS
Start State:
5 B 8
4 2 1
7 3 6

Goal State:
1 2 3
4 5 6
7 8 B

✗ FAILURE - Goal state not reached
Total states explored: 2
Execution time: 0.0001 seconds
Final heuristic value: 5
Best state reached:
5 2 8
4 B 1
7 3 6

S

Implementation Details and Analysis

Puzzle State Representation:

3×3 grid with blank tile 'B'

Efficient state comparison and hashing

Heuristic Functions:

h₁(n): Misplaced tiles count

h₂(n): Manhattan distance

Hill Climbing Features:

Steepest-ascent variant

Maximum iterations limit

Local optima detection

Simulated Annealing Features:

Exponential cooling schedule

Probabilistic acceptance of worse states

Temperature-based exploration

Theoretical Analysis:
Hill Climbing Limitations:

Gets stuck in local optima where no immediate neighbor improves the solution

Cannot escape plateaus or ridges in the search space

Highly sensitive to initial state

Simulated Annealing Advantages:

Probabilistic Acceptance: Can accept worse solutions early in search

Cooling Schedule: Gradually reduces exploration probability

Global Search: Continues searching even after finding local improvements

Expected Performance Differences:
Success Rate: SA should achieve higher success rates

Solution Quality: SA typically finds better solutions

Exploration: SA explores more states but finds better paths

Robustness: SA is less sensitive to initial conditions

Algorithm Comparison Factors:
Convergence: HC converges faster but to local optima; SA converges slower but to better solutions

Efficiency: HC uses fewer resources per iteration; SA requires more computation

Robustness: SA handles rugged search spaces better

Parameter Sensitivity: SA requires careful tuning of temperature and cooling

The implementation includes comprehensive experimentation with different heuristics and random seeds to provide statistically meaningful comparisons between the two algorithms.